Решения должны быть реализованы в файле *Homework/02/solution.py*

# Задание 1: реализация self-attention(2 балла)

В данном задании вам предстоит самостоятельно реализовать механизм self-attention(пока не multihead-версию). Суть проста:
- даны 3 тензора $ Q, K, V $ с размерами $ (BatchSize, SeqLength, HiddenDim) $
- необходимо вычислить attention scores, применив softmax к результату $ \frac{QK^{T}}{\sqrt(HiddenDim)} $
- вычислить attention output, умножив V на attention scores

Входная и выходная размерности совпадают!

Решение реализуется в функции compute_attention

# Задание 2: реализация multihead self-attention(3 балла)

Обобщим предыдущий подход на случай, когда мы работаем с несколькими головами:

- даны 3 тензора $Q, K, V$ с размерами $ (BatchSize, NHeads, SeqLength, DimPerHead) $ и квадратная матрица $O$ размера $ (NHeads * DimPerHead, NHeads * DimPerHead) $
- нунжно посчитать self-attention для каждой головы
- сконкатенировать результаты предыдущего пункта в один тензор
- применить к предыдущему пункту матрицу O

Входная и выходная размерности совпадают!

Решение реализуется в функции compute_multihead_attention

# Задание 3: реализация RoPE(Rotary Position Embedding)(5 баллов)

На лекции мы обсудили один из вариантов внедрения позиционной информации в представление токена- аддитивный, в рамках которого к эмбеддингу каждого токена $ x_{i} $ прибавляется позиционный эмбеддинг $ p_{i} $, соответствующий позиции этого токена. Такой эмбеддинг называется абсолютным, т.к. при его вычислении мы работаем с фиксированной позицией токена(проще говоря, при вычислении мы учитываем только один индекс, связанный с входной последовательностью). Затем оба эмбеддинга можно использовать при вычислении query/key/value-векторов:
$$ f_{\{q, k, v\}}(x_{i}, i) = W_{\{q, k, v\}} \cdot (x_{i} + p_{i}) $$

В этом задании вам предлагается реализовать другой сценарий внедрения позиционной информации в представление токена- RoPE, который характеризуется:
- мультипликативностью(вместо сложения эмбеддинга токена и вектора его позиционного представления мы будем умножать эмбеддинг токена на некую матрицу)
- относительностью(вместо фиксированной позиции токена мы будем работать с двумя индексами i, j, которые будут определять расстояние между токенами под соответствующими индексами)

При использовании данного подхода модели быстрее сходятся и дают лучшее качество в практических задачах.

## Как это работает
Вспомним, что в механизме attention мы считаем сходство между токенами i и j следующим образом: пусть $ q_{i} = f_{q}(x_{i}, i)$ и $ k_{j}=f_{k}(x_{i}, j)$. Тогда $$ sim(i, j) = <f_{q}(x_{i}, i), f_{k}(x_{i}, j)> $$ Мы хотим найти функцию $ g $  такую, что $$ <f_{q}(x_{i}, i), f_{k}(x_{j}, j)>  = g(x_{i}, x_{j}, i-j),$$ т.е. такую, чтобы при подсчёте она учитывала только НЕпозиционные эмбеддинги для каждого токена и расстояние между этими двумя токенами, но при этом её значение вычислялось через скалярное произведение 2 векторов. Если размерность наших векторов- 2, то нашу задачу можно решить через матрицу вращения. Пусть $$ f_{q}(x_{m}, m) = \begin{pmatrix} cos(m \theta) & -sin(m \theta) \\ sin(m \theta) & cos(m \theta) \end{pmatrix} \begin{pmatrix} W_{1,1} & W_{1,2} \\ W_{2,1} & W_{2,2} \end{pmatrix} \begin{pmatrix} x_{1}^{m} \\ x_{2}^{m} \end{pmatrix} = R_{m} W x_{m},$$ где $R_{m}$- матрица вращения. Тогда $$ <f_{q}(x_{i}, i), f_{k}(x_{j}, j)> = (R_{i} W x_{i})^{T}(R_{j} W x_{j}) = x_{i}^{T}W_{q}R_{i-j}W_{k}x_{j} = g(x_{i}, x_{j}, i-j)$$

Обобщение решения задачи в многомерном будет работать так:
1. все наши векторы имеют чётную размерность d
2. матрица вращения будет иметь вид

![rotation_matrix](data/rotation_matrix.jpeg)

Последняя деталь пазла- это определение вычисления $ \theta_{i} $ в матрице вращения: $$ \theta_{i} = 10000 ^ {-2(i-1) /d}, i \in {1, 2, ..., d / 2} $$ 

## Ваша задача
Реализовать добавление позиционной информации с помощью RoPE. В нашем случае это будет означать, что каждый вектор $ x_{m} $ будет умножаться на матрицу $ R_{\theta, m}^{d} $. На вход вашему решению будет приходить тензор $ x $ размера $ (BatchSize, SeqLength, NHeads, DimPerHead) .$ За более подробной информацией по интуиции подхода и деталям реализации можно и нужно обращаться к [статье](https://arxiv.org/abs/2104.09864).

Входная и выходная размерности совпадают!

Решение реализуется в функции compute_rotary_embeddings